# This project aims to go through messy data, and how to make it clean 

In [2]:
# Importing necessary Libraries 
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import warnings
warnings.filterwarnings('ignore')

In [3]:
# Importing survey data
Survey_df = pd.read_csv('Ask A Manager Salary Survey 2021 (Form Responses).csv')

In [4]:
# Overview of the data (first 5 rows)
Survey_df.head()

,Timestamp,How old are you?,What industry do you work in?,Job title,"If your job title needs additional context, please clarify here:","What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)","How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.",Please indicate the currency,"If ""Other,"" please indicate the currency here:","If your income needs additional context, please provide it here:",What country do you work in?,"If you're in the U.S., what state do you work in?",What city do you work in?,How many years of professional work experience do you have overall?,How many years of professional work experience do you have in your field?,What is your highest level of education completed?,What is your gender?,What is your race? (Choose all that apply.)
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,"62,000",3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,"60,000",7000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White


<hr style="border:1px solid gray"></hr>

### First things first, relabeling attributes (Survey Questions)
>**timestamp:** Timestamp
>
>**age:** How old are you?
>
>**industry:** What industry do you work in?
>
>**job_title:** Job title
>
>**jt_add_context:** If your job title needs additional context, please clarify here:
>
>**annual_salary:** What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.) 
>
>**incentives:** How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.
>
>**currency:** Please indicate the currency
>
>**other_currency:** If "Other," please indicate the currency here: 
>
>**income_add_context:** If your income needs additional context, please provide it here:
>
>**workin_country:** What country do you work in?
>
>**ifUS_state:** If you're in the U.S., what state do you work in?
>
>**workin_city:** What city do you work in?
>
>**overall_workexp:** How many years of professional work experience do you have overall?
>
>**field_workexp:** How many years of professional work experience do you have in your field?
>
>**highest_education:** What is your highest level of education completed?
>
>**gender:** What is your gender?
>
>**race:** What is your race? (Choose all that apply.)

In [5]:
# Defining a list with new attributes' names
columns = ['timestamp', 'age', 'industry', 'job_title', 'jt_add_context', 'annual_salary', 'incentives',
           'currency', 'other_currency', 'income_add_context', 'workin_country', 'ifUS_state', 'workin_city',
           'overall_workexp', 'field_workexp', 'highest_education', 'gender', 'race']

In [6]:
# Replacing old, questionnaire, attributes with new ones
Survey_df.columns = columns

<hr style="border:1px solid gray"></hr>

### Now, lets discover our survey answers

In [7]:
# Shape of our data (records, attributes)
Survey_df.shape

(27844, 18)

In [8]:
# Overview of the data (first 5 rows)
Survey_df.head()

,timestamp,age,industry,job_title,jt_add_context,annual_salary,incentives,currency,other_currency,income_add_context,workin_country,ifUS_state,workin_city,overall_workexp,field_workexp,highest_education,gender,race
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,"62,000",3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,"60,000",7000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White


In [9]:
# Overviewing data's information
Survey_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27844 entries, 0 to 27843
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   timestamp           27844 non-null  object 
 1   age                 27844 non-null  object 
 2   industry            27774 non-null  object 
 3   job_title           27844 non-null  object 
 4   jt_add_context      7202 non-null   object 
 5   annual_salary       27844 non-null  object 
 6   incentives          20632 non-null  float64
 7   currency            27844 non-null  object 
 8   other_currency      188 non-null    object 
 9   income_add_context  3024 non-null   object 
 10  workin_country      27844 non-null  object 
 11  ifUS_state          22892 non-null  object 
 12  workin_city         27766 non-null  object 
 13  overall_workexp     27844 non-null  object 
 14  field_workexp       27844 non-null  object 
 15  highest_education   27634 non-null  object 
 16  gend

In [10]:
# Discovering the number of missing values for each attribute
np.sum(Survey_df.isna())

timestamp                 0
age                       0
industry                 70
job_title                 0
jt_add_context        20642
annual_salary             0
incentives             7212
currency                  0
other_currency        27656
income_add_context    24820
workin_country            0
ifUS_state             4952
workin_city              78
overall_workexp           0
field_workexp             0
highest_education       210
gender                  165
race                    164
dtype: int64

In [11]:
# Checking for number of duplicate records
np.sum(Survey_df.duplicated())

# there is no duplicates

0

## Fixing data types


In [12]:
# Defining a dict. with attribute_name as keys, and new dtype as values
Fixed_Dtypes = {'age': 'category', 'annual_salary': 'int64', 'overall_workexp': 'category',
                'field_workexp': 'category', 'highest_education': 'category', 'gender': 'category'}

In [13]:
# Overview of the annual salary attribute (first 5 rows)
Survey_df['annual_salary'].head()

0    55,000
1    54,600
2    34,000
3    62,000
4    60,000
Name: annual_salary, dtype: object

In [14]:
# Replacing all commas in each annual salary value with nothing
Survey_df['annual_salary'] = Survey_df['annual_salary'].str.replace(',', '')
Survey_df['annual_salary'].head()

0    55000
1    54600
2    34000
3    62000
4    60000
Name: annual_salary, dtype: object

In [15]:
# Casting the previously identified dict.
Survey_df = Survey_df.astype(Fixed_Dtypes)

In [16]:
# Overviewing data's information
Survey_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27844 entries, 0 to 27843
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   timestamp           27844 non-null  object  
 1   age                 27844 non-null  category
 2   industry            27774 non-null  object  
 3   job_title           27844 non-null  object  
 4   jt_add_context      7202 non-null   object  
 5   annual_salary       27844 non-null  int64   
 6   incentives          20632 non-null  float64 
 7   currency            27844 non-null  object  
 8   other_currency      188 non-null    object  
 9   income_add_context  3024 non-null   object  
 10  workin_country      27844 non-null  object  
 11  ifUS_state          22892 non-null  object  
 12  workin_city         27766 non-null  object  
 13  overall_workexp     27844 non-null  category
 14  field_workexp       27844 non-null  category
 15  highest_education   27634 non-null  

In [17]:
# Casting 'timestamp' into datetime64 data type
Survey_df['timestamp'] = pd.to_datetime(Survey_df['timestamp'])

In [18]:
# Lets check data info. again
Survey_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27844 entries, 0 to 27843
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   timestamp           27844 non-null  datetime64[ns]
 1   age                 27844 non-null  category      
 2   industry            27774 non-null  object        
 3   job_title           27844 non-null  object        
 4   jt_add_context      7202 non-null   object        
 5   annual_salary       27844 non-null  int64         
 6   incentives          20632 non-null  float64       
 7   currency            27844 non-null  object        
 8   other_currency      188 non-null    object        
 9   income_add_context  3024 non-null   object        
 10  workin_country      27844 non-null  object        
 11  ifUS_state          22892 non-null  object        
 12  workin_city         27766 non-null  object        
 13  overall_workexp     27844 non-null  category  

___________________________________________________________________________________________

## missing values

In [19]:
# Lets check industry rows with missing values
Industry_null = Survey_df.loc[Survey_df['industry'].isna()]
Industry_null.head()

,timestamp,age,industry,job_title,jt_add_context,annual_salary,incentives,currency,other_currency,income_add_context,workin_country,ifUS_state,workin_city,overall_workexp,field_workexp,highest_education,gender,race
360,2021-04-27 11:08:41,45-54,NaN,Proposal Manager,NaN,87938,4000.0,USD,NaN,NaN,US,North Carolina,Wilmington,21 - 30 years,11 - 20 years,College degree,Woman,White
645,2021-04-27 11:12:59,35-44,NaN,Legal editor,NaN,82000,8200.0,USD,NaN,NaN,USA,New York,Rochester,11 - 20 years,11 - 20 years,Master's degree,Woman,White
1604,2021-04-27 11:28:20,25-34,NaN,Commissioning Editor,NaN,36000,NaN,GBP,NaN,NaN,UK,NaN,London,11 - 20 years,8 - 10 years,Master's degree,Woman,"Asian or Asian American, White"
2055,2021-04-27 11:37:01,35-44,NaN,Research Manager,NaN,115000,NaN,USD,NaN,NaN,US,District of Columbia,DC,21 - 30 years,11 - 20 years,Master's degree,Woman,White
2063,2021-04-27 11:37:12,35-44,NaN,office manager,manage a building vs managing employees,40000,NaN,USD,NaN,NaN,USA,Nevada,Las Vegas,11 - 20 years,11 - 20 years,High School,Woman,White


*Maybe if we impute the missing values according to associated job_title....*

In [20]:
# Unique job title values for industry nulls
Industry_null['job_title'].unique()

array(['Proposal Manager', 'Legal editor', 'Commissioning Editor ',
       'Research Manager', 'office manager', 'Research Analyst',
       'Senior Regulatory Affairs Specialist', 'Adult Services Librarian',
       'Program Manager ', 'GIS Analyst', 'Director of Compliance', 'EA',
       'Research Scientist', 'Youth Services Librarian',
       'Financial Analyst (part-qualified)', 'Executive Assistant II',
       'Office Supervisor ', 'Chief of Staff, Customer Experience',
       'Veterinarian ', 'Lab Assistant (Level II)',
       'Customer Service Supervisor', 'Customer Service analyst ',
       'Manager, strategy', 'People Operations Manager ',
       'VP of Finance & Operations', 'Office manager ', 'UX Designer',
       'Senior Manager of Ops Strategy', 'Credit Supervisor ',
       'Operations Manager', 'hr manager', 'Director of HR',
       'Accounting manager', 'HR Generalist II',
       'Campaign and Program Manager ', 'Director',
       'Marketing Coordinator', 'Vendor Manager',

In [21]:
# No. of unique job title values for industry nulls
Industry_null['job_title'].nunique()

68

In [22]:
# No. of unique job title values for survey
Survey_df['job_title'].nunique()

14244

In [23]:
# Removing whitespaces at the beginning and the end of each string
Survey_df['job_title'] = Survey_df['job_title'].str.strip()
Survey_df['job_title'].nunique()

13316

In [24]:
# Converting strings into lowercase
Survey_df['job_title'] = Survey_df['job_title'].str.lower()
Survey_df['job_title'].nunique()

12035

In [25]:
# Initializing a bag of stopwords
stop = stopwords.words('english')

# Method for excluding stopwords from an attribute
def Exclude_Stopwords(attribute):
    # using list comprehension
    return attribute.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [26]:
# Initalize a lemmatizer
wn_lemm = WordNetLemmatizer()

# Method for lemmatizing an attriute
def Lemmatization(attribute):
    # using list comprehension
    return attribute.apply(lambda word: wn_lemm.lemmatize(word))

***

In [27]:
# Excluding stopwords using Exclude_Stopwords method
Survey_df['job_title'] = Exclude_Stopwords(Survey_df['job_title'])
Survey_df['job_title'].nunique()

11870

In [28]:
# Lemmatizing values
Survey_df['job_title'] = Lemmatization(Survey_df['job_title'])
Survey_df['job_title'].nunique()

11867

In [29]:
# Removing all special characters, using regex
Survey_df['job_title'] = Survey_df['job_title'].str.replace('[^\w\s]', '')
Survey_df['job_title'].nunique()

11867

__________________________________________________________________________

In [30]:
#fil industry nulls with not specified 
Survey_df['industry'].fillna("Not-Specified", inplace = True)

In [31]:
Survey_df['industry'].nunique()

1207

In [32]:
# Removing whitespaces at the beginning and the end of each string
Survey_df['industry'] = Survey_df['industry'].str.strip()

# Converting strings into lowercase
Survey_df['industry'] = Survey_df['industry'].str.lower()

# Excluding stopwords using Exclude_Stopwords method
Survey_df['industry'] = Exclude_Stopwords(Survey_df['industry'])

# Lemmatizing values
Survey_df['industry'] = Lemmatization(Survey_df['industry'])

# Removing all special characters, using regex
Survey_df['industry'] = Survey_df['industry'].str.replace('[^\w\s]', '')

Survey_df['industry'].nunique()

988

In [33]:
# Industry substrings to search for
SubStrings = ['nonprofit', 'librar', 'biotech', 'academi', 'education', 'aerospace', 'pharma', 'food', 'archit',
              'real estate', 'environment', 'vet', 'construction', 'sale', 'marketing', 'research', 'energy', 'gov']

# Strings to be mapped to
MaptoStrings = ['nonprofit', 'library', 'biotech', 'academia', 'education', 'aerospace', 'pharma', 'food services',
                'architecture', 'real estate', 'environment', 'veterinary', 'construction', 'sales', 'marketing',
                'research', 'energy', 'government']

In [34]:
# Method to map all rows with specific substring in a certain column value with a new one
def ReplaceColumnValue(df, column, SubStrings, MaptoStrings):
    for (ss, sm) in zip(SubStrings, MaptoStrings):
        # Filtering rows containing specific substring
        contain_substring = df[df[column].str.contains(ss)]
        # Replacing all rows certain column values with new one
        df.loc[contain_substring.index, column] = sm

In [35]:
# Mapping old 'industry' values to new ones
ReplaceColumnValue(Survey_df, 'industry', SubStrings, MaptoStrings)
Survey_df['industry'].nunique()

600

In [36]:
# Filling missing values in both columns with 'Not-Specified'
Survey_df['jt_add_context'].fillna("Not-Specified", inplace = True)
Survey_df['income_add_context'].fillna("Not-Specified", inplace = True)

In [37]:
# Filling missing values with 0 (as it is float64)
Survey_df['incentives'].fillna(0, inplace = True)

In [38]:
Survey_df['currency'].nunique()

11

In [39]:
Survey_df['other_currency'].nunique()

112

In [40]:
# Removing whitespaces at the beginning and the end of each string, for both attributes
Survey_df['currency'] = Survey_df['currency'].str.strip()
Survey_df['other_currency'] = Survey_df['other_currency'].str.strip()

# Converting strings into lowercase, for both attributes
Survey_df['currency'] = Survey_df['currency'].str.lower()
Survey_df['other_currency'] = Survey_df['other_currency'].str.lower()

In [41]:
# Dataframe with rows didn't choose 'other'
No_Other = Survey_df.loc[Survey_df['currency'] != 'other']

# Validating if number of rows = number of nulls
len(No_Other.index) == np.sum(No_Other['other_currency'].isna())

False

In [42]:
# Filtering no null responses
No_Other.loc[No_Other['other_currency'].notnull()].head()

,timestamp,age,industry,job_title,jt_add_context,annual_salary,incentives,currency,other_currency,income_add_context,workin_country,ifUS_state,workin_city,overall_workexp,field_workexp,highest_education,gender,race
752,2021-04-27 11:14:29,45-54,library,administrative librarian,I run a branch library and am responsible for ...,76302,0.0,usd,"$76,302.34",Not-Specified,USA,Arizona,Glendale,21 - 30 years,21 - 30 years,Master's degree,Woman,White
766,2021-04-27 11:14:39,25-34,recruitment hr,lead payroll & benefits specialist,Not-Specified,64000,6400.0,usd,my bonus is based on performance up to 10% of ...,Not-Specified,US,New York,Schenectady,5-7 years,5-7 years,College degree,Woman,White
776,2021-04-27 11:14:45,45-54,education,sr technical support rep,I,53000,0.0,usd,"i work for an online state university, managin...","I have excellent health benefits, time off, si...",USA,New Jersey,Trenton,8 - 10 years,2 - 4 years,Master's degree,Woman,White
1001,2021-04-27 11:18:27,35-44,recruitment hr,hr manager/accounts payable,Not-Specified,53500,0.0,usd,0,Not-Specified,US,Minnesota,Minneapolis,11 - 20 years,2 - 4 years,College degree,Woman,White
1915,2021-04-27 11:34:43,35-44,education,instructional coach,Not-Specified,13560,0.0,usd,kwd,International overseas hires also get flights ...,Kuwait,NaN,Hawalli,11 - 20 years,1 year or less,Master's degree,Woman,White


*These are unrelated values, and should be fixed into null*

In [43]:
# Extracting not-null rows indices
To_null_indices = No_Other.loc[No_Other['other_currency'].notnull()].index

# Converting these values into NaN, in Survey_df
for index in To_null_indices:
    Survey_df['other_currency'][index] = np.nan

# Validating one more time
No_Other = Survey_df.loc[Survey_df['currency'] != 'other']
len(No_Other.index) == np.sum(No_Other['other_currency'].isna())

True

In [44]:
# Dataframe with rows did choose 'other'
Other = Survey_df.loc[Survey_df['currency'] == 'other']

Other['other_currency'].nunique()

70

In [45]:
# Currency code to be mapped to
MapToCurr = ['ars', 'eur', 'usd', 'aud/nzd', 'brl', 'cny', 'dkk', 'czk', 'php',
             'inr', 'krw', 'nok', 'mxn', 'pln', 'sgd', 'thb', 'myr', 'NaN']

# List of keywords to be mapped to corresponding currency code
ToBeMapped = [['argentin', 'ars'], ['eur', 'euro'], ['american', 'usd', 'us dollar'], ['australian', 'aud', 'nzd'],
              ['brl', 'br$'], ['china', 'rmb', 'cny'], ['danish', 'dkk'], ['czech', 'czk'], ['philippine', 'php'],
              ['indian', 'rupees', 'inr'], ['korean', 'krw'], ['norwegian', 'nok'], ['mexican', 'mxn'],
              ['polish', 'zwoty', 'pln'], ['singapore', 'sgd'], ['taiwanese', 'thai', 'ntd', 'thb'], ['rm', 'myr'],
              ['israeli', 'ils', 'nis']]

In [46]:
# Method to map all rows with keyword in a certain column value with a new one
def ReplaceColumnValue_II(df, column, ToBeMapped, MapToCurr):
    for (ss, cm) in zip(ToBeMapped, MapToCurr):
        # for each keyword in current list
        for s in ss:
            # Filtering rows containing specific substring
            contain_substring = df[df[column].str.contains(s)]
            # Replacing all rows certain column values with new one
            df.loc[contain_substring.index, column] = cm

In [47]:
# Temporary dataframe with no-null values
temp_df = Survey_df.loc[Survey_df['other_currency'].notnull()]
# Using ReplaceColumnValue_II method to map values
ReplaceColumnValue_II(temp_df, 'other_currency', ToBeMapped, MapToCurr)

In [48]:
Survey_df['other_currency'] = temp_df['other_currency']
Survey_df['other_currency'].nunique()

32

In [49]:
# Filling na in order to correctly concat columns
Survey_df['other_currency'].fillna('-', inplace = True)

# Removing 'other' option
Survey_df['currency'].replace('other', '-', inplace = True)

In [50]:
# Concatenating both columns
Survey_df['updated_currency'] = Survey_df['currency'] + Survey_df['other_currency']

# Removing all hyphens
Survey_df['updated_currency'] = Survey_df['updated_currency'].str.strip('-')

# Dropping both columns
Survey_df.drop(['currency', 'other_currency'], axis = 1, inplace = True)

# Reordering dataframe column
Survey_df = Survey_df[['timestamp', 'age', 'industry', 'job_title', 'jt_add_context', 'annual_salary', 'incentives',
                       'updated_currency', 'income_add_context', 'workin_country', 'ifUS_state', 'workin_city', 
                       'overall_workexp', 'field_workexp', 'highest_education', 'gender', 'race']]

# Currency code should be uppercase, why did I change to lower?
Survey_df['updated_currency'] = Survey_df['updated_currency'].str.upper()

In [51]:
Survey_df['updated_currency'].unique()

array(['USD', 'GBP', 'CAD', 'EUR', 'AUD/NZD', 'INR', 'ARS', 'CHF', 'MYR',
       'ZAR', 'SEK', 'HKD', 'NOK', 'BR$', 'DKK', 'TTD', 'BRL', 'MXN',
       'CZK', 'BDT', 'PHP', 'PLN', 'TRY', 'CNY', 'NAN', '', 'JPY', 'SGD',
       'KRW', 'THB', 'IDR', 'LKR', 'EQUITY', 'SAR', 'CROATIAN KUNA'],
      dtype=object)

In [52]:
# Attribute bath-time (Strip, lower)
Survey_df['workin_country'] = Survey_df['workin_country'].str.strip()
Survey_df['workin_city'] = Survey_df['workin_city'].str.strip()

Survey_df['workin_country'] = Survey_df['workin_country'].str.lower()
Survey_df['workin_city'] = Survey_df['workin_city'].str.lower()

In [53]:
Survey_df['workin_country'].unique()

array(['united states', 'united kingdom', 'us', 'usa', 'canada', 'uk',
       'scotland', 'u.s.', 'the netherlands', 'australia', 'spain',
       'england', 'finland', 'united states of america', 'france',
       'germany', 'ireland', 'india', 'u.s>', 'isa', 'argentina',
       'great britain', 'united state', 'u.s.a', 'denmark', 'u.s.a.',
       'america', 'netherlands', 'switzerland', 'bermuda',
       'the united states', 'united state of america', 'malaysia',
       'mexico', 'united stated', 'south africa', 'belgium',
       'northern ireland', 'sweden', 'hong kong', 'kuwait', 'norway',
       'sri lanka', 'contracts', 'usa-- virgin islands', 'united statws',
       'england/uk', 'u.s',
       "we don't get raises, we get quarterly bonuses, but they periodically asses income in the area you work, so i got a raise because a 3rd party assessment showed i was paid too little for the area we were located",
       'unites states', 'england, uk.', 'greece', 'japan', 'u. s.',
       'bri

In [54]:
Survey_df['workin_country'].nunique()

251

In [55]:
Survey_df['workin_country'] = Exclude_Stopwords(Survey_df['workin_country'])
Survey_df['workin_country'].nunique()

243

In [56]:
# Countries to be mapped to
MapToCountry = ['Remote', 'USA', 'UK', 'Canada', 'UAE', 'Australia', 'New Zealand', 'China',
                'Brazil', 'Argentina', 'Germany', 'Spain', 'Italy', 'Japan', 'Czeck Republic',
                'Luxembourg', 'Denmark', 'Netherlands', 'Mexico', 'Romania', 'Palestine']

# List of keywords to be mapped to corresponding currency code
ToBeMappedCountry = [['remote'], ['state', 'united s', 'usa', 'u.s', 'america', 'u. s.', 'us gov','isa', 'i.s.', 'usd', 'califo', 'virginia', 'contracts', 'policy', 'bonus', 'finance', 'year', 'salary'],
                     ['uk', 'united k', 'u.k', 'britain', 'kingdom', 'englan', 'wales', 'scotland', 'ireland'], ['canad', 'canda', 'csnada'],
                     ['emirates', 'united', 'uae',  'u.a.'], ['australi'], ['new z', 'nz'], ['china', 'hong'], ['brasil', 'brazil'],
                     ['argentina'], ['germany'], ['spain', 'catalonia'], ['italy'], ['japan'], ['czech'], ['luxemb'],
                     ['danmark', 'denmark'], ['netherlands', 'nederland', 'nl'], ['xico'], ['romania'], ['israel']]

In [57]:
# Using ReplaceColumnValue_II method to map values
ReplaceColumnValue_II(Survey_df, 'workin_country', ToBeMappedCountry, MapToCountry)
# Replacing us with USA
Survey_df['workin_country'].replace('us', 'USA', inplace = True)

In [58]:
# Capitalize first letter
Survey_df['workin_country'] = Survey_df['workin_country'].str.capitalize()

Survey_df['workin_country'].nunique()

98

In [59]:
Survey_df['workin_country'].unique()

array(['Usa', 'Uk', 'Canada', 'Netherlands', 'Australia', 'Spain',
       'France', 'Germany', 'India', 'Argentina', 'Denmark',
       'Switzerland', 'Bermuda', 'Malaysia', 'Mexico', 'South africa',
       'Belgium', 'Sweden', 'China', 'Uae', 'Norway', 'Sri lanka',
       'Greece', 'Japan', 'Austria', 'Brazil', 'Global', 'Hungary',
       'Luxembourg', 'Colombia', 'New zealand', 'Trinidad tobago',
       'Cayman islands', '', 'Czeck republic', 'Latvia', 'Puerto rico',
       'Rwanda', 'Bangladesh', 'Romania', 'Serbia', 'Philippines',
       'Poland', 'Uxz', 'Italy', 'Jersey, channel islands', 'Afghanistan',
       'Palestine', 'Hartford', 'Taiwan', 'Cambodia', 'Vietnam', 'Remote',
       'Singapore', 'South korea', 'Thailand', 'Lithuania', 'Eritrea',
       'Indonesia', 'Cuba', 'Slovenia', "Cote d'ivoire", 'Somalia',
       'Slovakia', 'Sierra leone', 'International', 'Bahamas',
       'Costa rica', 'Chile', 'Qatar', 'Nigeria', 'Panamá', 'U.', 'Congo',
       'Uruguay', 'Pakistan', 'Ma

In [60]:
# Dataframe with respondants not working in th USA
Not_USA = Survey_df.loc[Survey_df['workin_country'] != 'Usa']
# Validating if number of rows = number of nulls
len(Not_USA.index) == np.sum(No_Other['ifUS_state'].isna())

False

In [61]:
# Filtering no-null answers
Not_USA.loc[Not_USA['ifUS_state'].notnull()].head()

,timestamp,age,industry,job_title,jt_add_context,annual_salary,incentives,updated_currency,income_add_context,workin_country,ifUS_state,workin_city,overall_workexp,field_workexp,highest_education,gender,race
4739,2021-04-27 12:45:35,25-34,law,funding coordinator,I'm basically an Admin at a law firm. I answer...,40000,1000.0,USD,I began working for this firm in 2014. I left ...,Canada,California,"company is in glendale, california i am in bar...",11 - 20 years,8 - 10 years,Master's degree,Woman,White
5017,2021-04-27 12:54:33,35-44,marketing,"vice president, marketing",Not-Specified,198000,5000.0,USD,Not-Specified,Uae,California,irvine,21 - 30 years,21 - 30 years,College degree,Woman,White
7789,2021-04-27 15:12:59,25-34,government,business industry specialist,CFIUS Case Manager,103690,0.0,USD,Not-Specified,Uxz,District of Columbia,washington dc,5-7 years,2 - 4 years,Master's degree,Woman,White
9425,2021-04-27 17:54:24,45-54,hospitality & events,event coordinator,Not-Specified,79612,0.0,CAD,Not-Specified,Canada,West Virginia,"edmonton, alberta",21 - 30 years,21 - 30 years,College degree,Woman,White
9701,2021-04-27 18:38:27,35-44,government,staff development specialist,Not-Specified,91000,0.0,USD,Not-Specified,Hartford,Connecticut,hartford,11 - 20 years,2 - 4 years,Master's degree,Woman,White


In [62]:
# Extracting not-null rows indices
To_null_indices = Not_USA.loc[Not_USA['ifUS_state'].notnull()].index

# Converting these values into NaN, in Survey_df
for index in To_null_indices:
    Survey_df['ifUS_state'][index] = np.nan

In [63]:
# Validating one more time
Not_USA = Survey_df.loc[Survey_df['workin_country'] != 'Usa']
len(Not_USA.index) == np.sum(Survey_df['ifUS_state'].isna())

False

In [64]:
# Comparing No. of respondants that work outside USA with no. of nulls for them in 'ifUS_state'
# To confirm that other nulls exist for respondants working in USA
len(Not_USA.index) == np.sum(Not_USA['ifUS_state'].isna())

True

In [65]:
# Dataframe with respondants working in USA
In_USA = Survey_df.loc[Survey_df['workin_country'] == 'Usa']
# Checking no. of nulls in 'ifUS_state'
np.sum(In_USA['ifUS_state'].isna())

177

In [66]:
# Filling null responses
Survey_df['ifUS_state'].fillna('Not-in-Usa', inplace = True)

In [67]:
# Filling null responses
Survey_df['workin_city'].fillna('Not-Specified', inplace = True)

In [68]:
# Adding 'Not-Specified' category to categorical variable
Survey_df['highest_education'] = Survey_df['highest_education'].cat.add_categories('Not-Specifed')

# Filling null responses
Survey_df['highest_education'].fillna(Survey_df['highest_education'].cat.categories[6], inplace = True)

In [69]:
# Dropping (removing) 'race' attribute
Survey_df.drop('race', inplace=True, axis=1)

In [70]:
# Adding 'Not-specifed' category to categorical variable
Survey_df['gender'] = Survey_df['gender'].cat.add_categories('Not-Specifed')
# Gender Categories
Gender_Cat = Survey_df['gender'].cat.categories
Gender_Cat

Index(['Man', 'Non-binary', 'Other or prefer not to answer',
       'Prefer not to answer', 'Woman', 'Not-Specifed'],
      dtype='object')

In [71]:
# Fixing 'gender' attribute
Survey_df['gender'].replace([Gender_Cat[1], Gender_Cat[2], Gender_Cat[3]], Gender_Cat[5], inplace = True)
Survey_df['gender'].fillna(Gender_Cat[5], inplace = True)

In [72]:
np.sum(Survey_df.isna())

timestamp             0
age                   0
industry              0
job_title             0
jt_add_context        0
annual_salary         0
incentives            0
updated_currency      0
income_add_context    0
workin_country        0
ifUS_state            0
workin_city           0
overall_workexp       0
field_workexp         0
highest_education     0
gender                0
dtype: int64

<hr style="border:1px solid gray"></hr>

In [73]:
# Save our clean data 
Survey_df.to_csv('Cleaned_Survey.csv')